"¿Cómo influyen los indicadores económicos y demográficos en la tasa de crecimiento de la población a nivel global?"

In [2]:
# Importar librerías necesarias
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [5]:
# Definir la estructura de la red bayesiana
LV_URB = "Nivel urbanización"
LV_EDU = "Nivel educativo"
RT_PARO = "Tasa de paro"
PNB = "Ingreso nacional bruto"
PIB = "Producto Interior Bruto"
GASTO_EDUCATIVO = "Gasto educativo"
ACCESO_SALUD = "Acceso a salud"
RT_FERTILIDAD = "Tasa de fertilidad"
RT_CRECIMIENTO = "Tasa de crecimiento de la población"
RT_MORT_INFANTIL = "Tasa de mortalidad infantil"

# Se le pasa por parámetro una lista de tuplas (origen --> destino)
model = BayesianNetwork()
model.add_nodes_from([LV_URB, LV_EDU, RT_PARO, PNB, PIB, GASTO_EDUCATIVO, 
                      ACCESO_SALUD, RT_FERTILIDAD, RT_CRECIMIENTO, RT_MORT_INFANTIL])

edges_RT_PARO = [(LV_EDU, RT_PARO), (LV_URB, RT_PARO)]
edges_PIB = [(RT_PARO, PIB), (PNB, PIB)]
edges_RT_FERTILIDAD = [(GASTO_EDUCATIVO, RT_FERTILIDAD), (ACCESO_SALUD, RT_FERTILIDAD)]
edges_RT_CRECIMIENTO = [(RT_FERTILIDAD, RT_CRECIMIENTO), (RT_MORT_INFANTIL, RT_CRECIMIENTO)]

model.add_edges_from(edges_RT_PARO)
model.add_edges_from(edges_PIB)
model.add_edge(PIB, GASTO_EDUCATIVO)
model.add_edges_from(edges_RT_FERTILIDAD)
model.add_edges_from(edges_RT_CRECIMIENTO)


In [18]:
# Suponiendo que ya tienes los datos para cada nodo, puedes definir los CPD
# Aquí, para simplificación, se usarán datos ficticios

# Ejemplo de cómo definir CPD (Distribución de Probabilidad Condicional)
cpd_tasa_fertilidad = TabularCPD(variable='Tasa de fertilidad', variable_card=2, values=[[0.6], [0.4]])
cpd_esperanza_vida = TabularCPD(variable='Esperanza de vida al nacer', variable_card=2, values=[[0.7], [0.3]])
cpd_tasa_mortalidad_infantil = TabularCPD(variable='Tasa de mortalidad infantil', variable_card=2, values=[[0.8], [0.2]])

cpd_tasa_crecimiento_poblacion = TabularCPD(
    variable='Tasa de crecimiento de la población', variable_card=2,
    values=[[0.9, 0.6, 0.5, 0.2, 0.7, 0.4, 0.1, 0.05],
            [0.1, 0.4, 0.5, 0.8, 0.3, 0.6, 0.9, 0.95]],
    evidence=['Tasa de fertilidad', 'Esperanza de vida al nacer', 'Tasa de mortalidad infantil'],
    evidence_card=[2, 2, 2]
)

cpd_tasa_desempleo = TabularCPD(variable='Tasa de desempleo', variable_card=2, values=[[0.6], [0.4]])
cpd_ingreso_nacional_bruto = TabularCPD(variable='Ingreso nacional bruto per cápita', variable_card=2, values=[[0.7], [0.3]])

cpd_pib_per_capita = TabularCPD(
    variable='PIB per cápita', variable_card=2,
    values=[[0.8, 0.5, 0.3, 0.1],
            [0.2, 0.5, 0.7, 0.9]],
    evidence=['Tasa de desempleo', 'Ingreso nacional bruto per cápita'],
    evidence_card=[2, 2]
)

cpd_gasto_educacion = TabularCPD(
    variable='Gasto en educación (% del PIB)', variable_card=2, 
    values=[[0.7, 0.3], 
            [0.3, 0.7]], 
    evidence=['PIB per cápita'], 
    evidence_card=[2])
cpd_acceso_salud = TabularCPD(variable='Acceso a servicios de salud', variable_card=2, values=[[0.6], [0.4]])

cpd_tasa_fertilidad_cond = TabularCPD(
    variable='Tasa de fertilidad', variable_card=2,
    values=[[0.9, 0.4, 0.3, 0.1],
            [0.1, 0.6, 0.7, 0.9]],
    evidence=['Gasto en educación (% del PIB)', 'Acceso a servicios de salud'],
    evidence_card=[2, 2]
)

cpd_esperanza_vida_cond = TabularCPD(
    variable='Esperanza de vida al nacer', variable_card=2,
    values=[[0.8, 0.5, 0.2, 0.1],
            [0.2, 0.5, 0.8, 0.9]],
    evidence=['Acceso a servicios de salud', 'Tasa de mortalidad infantil'],
    evidence_card=[2, 2]
)

cpd_nivel_urbanizacion = TabularCPD(variable='Nivel de urbanización', variable_card=2, values=[[0.7], [0.3]])
cpd_tasa_desempleo_cond = TabularCPD(
    variable='Tasa de desempleo', variable_card=2,
    values=[[0.6, 0.3],
            [0.4, 0.7]],
    evidence=['Nivel de urbanización'],
    evidence_card=[2]
)

In [7]:
# Añadir todos los CPD al modelo
model.add_cpds(cpd_tasa_fertilidad, cpd_esperanza_vida, cpd_tasa_mortalidad_infantil, 
               cpd_tasa_crecimiento_poblacion, cpd_tasa_desempleo, cpd_ingreso_nacional_bruto, 
               cpd_pib_per_capita, cpd_gasto_educacion, cpd_acceso_salud, cpd_tasa_fertilidad_cond, 
               cpd_esperanza_vida_cond, cpd_nivel_urbanizacion, cpd_tasa_desempleo_cond)

# Verificar que el modelo sea válido
assert model.check_model()

# Realizar inferencias
infer = VariableElimination(model)
result = infer.map_query(variables=['Tasa de crecimiento de la población'])
print(result)

Eliminating: Tasa de fertilidad: 100%|██████████| 9/9 [00:00<00:00, 574.10it/s]

{'Tasa de crecimiento de la población': 1}
